<a href="https://colab.research.google.com/github/issa-rashdan/INFORM_Project_M/blob/main/data_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/issa-rashdan/INFORM_Project_M.git

fatal: destination path 'INFORM_Project_M' already exists and is not an empty directory.


In [3]:
import numpy as np
from INFORM_Project_M.data.echosounder_data.load_data.get_echograms_2 import get_echograms, count_classes_in_echograms

# You can select a specific year or use 'all' to include multiple years.
years = 2014
minimum_shape = 224
tuple_frequencies = (18, 38, 70, 120, 200, 333)
echograms = get_echograms(years=years, tuple_frequencies=tuple_frequencies, minimum_shape=minimum_shape)
print(f"Number of echograms: {len(echograms)}")

Selected 10 echograms from year 2014
Number of echograms: 10


In [12]:
import os
import random
# Randomly split echograms into train/test
random.seed(42)  # Set a seed for reproducibility
Train_eg = []
test_eg = []
echogram = random.shuffle(echograms)  # Shuffle echograms randomly
for i,echogram in enumerate(echograms):
  if i < 0.8*len(echograms):
    Train_eg.append(echogram)
  else:
    test_eg.append(echogram)

#Save the echograms to files
train_dir = '/content/INFORM_Project_M/train'
test_dir = '/content/INFORM_Project_M/test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for i, echogram in enumerate(Train_eg):
 os.path.join(train_dir, f'echogram_{i}.npy')
for i, echogram in enumerate(test_eg):
  os.path.join(test_dir, f'echogram_{i}.npy')

# Print the number of echograms in each set
print('Train Echogram:', len(Train_eg))
print('Test Echogram:', len(test_eg))

Train Echogram: 8
Test Echogram: 2


In [9]:
import os, random
import numpy as np
from PIL import Image

# Directories
train_dir = "/content/INFORM_Project_M/train"
test_dir  = "/content/INFORM_Project_M/test"
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Train/test split
random.seed(42)
random.shuffle(echograms)
split_idx = int(0.8 * len(echograms))
train_egs, test_egs = echograms[:split_idx], echograms[split_idx:]

# Helper: convert Echogram → PIL
def echogram_to_pil(e):
    data = e.data_memmap()  # <— if not correct, tell me the right accessor
    if isinstance(data, dict):     # likely {freq: 2D array}
        arr = np.array(list(data.values())[0])  # just take one frequency
    else:
        arr = np.asarray(data)
    arr = (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr) + 1e-8)
    arr = (arr * 255).astype(np.uint8)
    return Image.fromarray(arr)

# Save echograms
for i, eg in enumerate(train_egs):
    echogram_to_pil(eg).save(os.path.join(train_dir, f"echogram_{i:05d}.png"))

for i, eg in enumerate(test_egs):
    echogram_to_pil(eg).save(os.path.join(test_dir, f"echogram_{i:05d}.png"))

print("Train Echograms:", len(train_egs))
print("Test Echograms:", len(test_egs))


AttributeError: 'Echogram' object has no attribute 'data_memmap'